In [3]:
from tensorflow import keras 
from keras import layers, applications
# from keras.preprocessing import image
from keras.applications.efficientnet import EfficientNetB0, EfficientNetB5, EfficientNetB7
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.densenet import DenseNet121
# from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input

import os 
import numpy as np
import cv2
from keras.models import Sequential
import pandas as pd 

# Data preprocessing 
from keras.preprocessing.image import load_img, img_to_array

In [4]:
# Load all the images from the folder 
dataFolder = os.path.dirname('data/x5_RGB/train/RGB/')
imagePaths = []
pil = []
classLabels = []

In [5]:

isIgnoredFile = lambda x: x[0] == "."

for case in os.listdir(dataFolder):
    # print(case)
    if isIgnoredFile(case):
        continue

    f = os.path.join(dataFolder, case)
    for image in os.listdir(f):
        if isIgnoredFile(image):
            continue
        label = f[-1]
        classLabels.append(label)
        imagePath = os.path.join(dataFolder, case, image)
        imagePaths.append(imagePath)

In [6]:
df = pd.DataFrame(data=zip(classLabels, imagePaths), columns=["Labels", "Images"])

In [7]:
print("The labels in the dataset are: ", set(df['Labels']))

The labels in the dataset are:  {'3', '5', '4', '2', '6', '1', '0'}


In [8]:
IMAGE_SIZE = 224 
images = []
for imagePath in imagePaths:
    img = cv2.imread(imagePath)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))   
    images.append(img)

# Convert the images to an np array 
images = np.array(images)
images = images.astype('float32')/255.0 #Normalize 
images.shape

(6000, 224, 224, 3)

In [9]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y =df['Labels'].values
yLabelEncoder = LabelEncoder ()
y = yLabelEncoder.fit_transform (y)

In [10]:
from sklearn.compose import ColumnTransformer
y=y.reshape(-1,1)

ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 0)	1.0
  (12, 0)	1.0
  (13, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  :	:
  (5940, 5)	1.0
  (5941, 5)	1.0
  (5942, 5)	1.0
  (5943, 5)	1.0
  (5944, 5)	1.0
  (5945, 5)	1.0
  (5946, 5)	1.0
  (5947, 5)	1.0
  (5948, 5)	1.0
  (5949, 5)	1.0
  (5950, 5)	1.0
  (5951, 5)	1.0
  (5952, 5)	1.0
  (5953, 5)	1.0
  (5954, 5)	1.0
  (5955, 5)	1.0
  (5956, 5)	1.0
  (5957, 5)	1.0
  (5958, 5)	1.0
  (5959, 5)	1.0
  (5960, 5)	1.0
  (5961, 5)	1.0
  (5962, 5)	1.0
  (5963, 5)	1.0
  (5964, 5)	1.0


In [11]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Shufflt the data around for better training 
images, Y = shuffle(images, Y, random_state=32)

TEST_SIZE = 0.05
xTrain, xTest, yTrain, yTest = train_test_split(images, Y, test_size=0.05, random_state=21)

# Verify the shapes of the training and testing data 
print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTest.shape)

(5700, 224, 224, 3)
(5700, 7)
(300, 224, 224, 3)
(300, 7)


In [12]:
# Training the NN 
NUM_CLASSES = 7

In [13]:

inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = inputs 
model = Sequential()
b0 = EfficientNetB0(weights="imagenet", include_top=False)
b5 = EfficientNetB5(weights="imagenet", include_top=False)  
model.add(b0)
denseb0b5 = layers.Dense(456)
model.add(b0)
model.add(denseb0b5)
model.add(b5)


Metal device set to: Apple M1


2022-02-28 14:20:12.584227: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-28 14:20:12.585553: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ValueError: Exception encountered when calling layer "normalization" (type Normalization).

Dimensions must be equal, but are 1280 and 3 for '{{node efficientnetb0/normalization/sub}} = Sub[T=DT_FLOAT](efficientnetb0/rescaling/add, efficientnetb0/normalization/sub/y)' with input shapes: [?,?,?,1280], [1,1,1,3].

Call arguments received:
  • inputs=tf.Tensor(shape=(None, None, None, 1280), dtype=float32)